Installing libraries

In [91]:
import stanza
import numpy as np
import pandas as pd
import re

In [92]:
import nltk
nltk.download('wordnet')
import re
from bs4 import BeautifulSoup
import os
import contractions
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('omw-1.4')

from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

import re
# import pickle
from emot.emo_unicode import UNICODE_EMOJI # For emojis
from emot.emo_unicode import EMOTICONS_EMO # For EMOTICONS

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Importing bad words vocab and dataset

In [93]:
bad_words=[]
with open('/kaggle/input/bad-bad-words/bad-words.csv') as f:
    for line in f.readlines():
        bad_words.append(line.split('\n')[0])

bad_words_set = set(bad_words)
dataset = pd.read_csv('/kaggle/input/comments/comments2.csv')

In [8]:
dataset

,Unnamed: 0,comments
0,0,A lot of the clients I work with use SVB. This...
1,1,Remember when Jim Cramer a month ago said on h...
2,2,"The FDIC only insures up to $250k, what happen..."
3,3,"This is a pretty big deal , they had a huge am..."
4,4,If anyone's wondering how a bank with 200+ bil...
...,...,...
65689,65689,They already have enough local staff to fill t...
65690,65690,Indeed it is! It still is ridiculously below t...
65691,65691,Amazon did because it's their play for DoD con...
65692,65692,"Thanks, I didn't get past the paywall."


In [9]:
def remove_urls(text):
    text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', text, flags=re.MULTILINE)
    return (text)

def remove_contractions(text) :
    expanded_words = []
    expanded_text = ''
    for word in text.split():
        expanded_words.append(contractions.fix(word))
        expanded_text = ' '.join(expanded_words)
    return expanded_text


remove_non_english = lambda s: re.sub(r'[^a-zA-z]', ' ', s)
remove_spaces = lambda s: re.sub(' +',' ', s)


In [10]:

def cleaning(text):
    #remove urls
    text = remove_urls(text)
    #remove html tags
    text = BeautifulSoup(text, "lxml").text
    #remove contractions 
    text = remove_contractions(text)
    #remove non-alphabetic chars 
    text = remove_non_english(text)
    #lowercase
    text = text.lower( )
    #remove extra spaces 
    text = remove_spaces(text)
    
    return text

In [14]:
dataset['cleaned_comments'] = list(map(cleaning, dataset.comments))

def convert_emojis(text):
    for emot in UNICODE_EMOJI:
        text = text.replace(emot, "_".join(UNICODE_EMOJI[emot].replace(",","").replace(":","").split()))
    return text

dataset['cleaned_comments'] = dataset['cleaned_comments'].apply(lambda row: convert_emojis(str(row)))

/opt/conda/lib/python3.7/site-packages/bs4/__init__.py:439: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  MarkupResemblesLocatorWarning


In [8]:
# from nltk.corpus import stopwords

# to_remove = ['not']
# new_stopwords = set(stopwords.words('english')).difference(to_remove)

# dataset['cleaned_comments'] = dataset['cleaned_comments'].apply(lambda x: " ".join(x for x in x.split() if x not in new_stopwords))

# from nltk.stem import WordNetLemmatizer
# lemmatizer = WordNetLemmatizer()

# dataset['cleaned_comments'] = dataset['cleaned_comments'].apply(lambda x: " ".join([lemmatizer.lemmatize(word) for word in x.split()]))


Importing Stanza POS tagger

In [11]:
pos_tagger = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos',use_gpu=True)

POS Tags for our dataset

In [15]:
dataset_sample = dataset[:].copy()

In [16]:
dataset_sample['cleaned_comments'][1]

'remember when jim cramer a month ago said on his show to buy it because it was cheap and had room to grow jimmy is such a meme '

In [75]:
words = []

In [76]:
pos_tags_dataset_sample =[]
ind = 1
for sentence in dataset_sample.iloc[:,2].values.tolist():
    if(ind%100==0):
        print(ind)
    ind+=1
    doc = pos_tagger(sentence)
#     pos_tags_dataset_sample.append([word.xpos for sent in doc.sentences for word in sent.words])
    words.append([word.text for sent in doc.sentences for word in sent.words])
    break

In [79]:
words = []
pos_tags_dataset = []
ind = 1
for sentence in dataset_sample.iloc[:,2].values.tolist():
    if(ind%1000==0):
        print(ind)
    ind+=1
    doc = pos_tagger(sentence)
    pos_tags_dataset.append([word.xpos for sent in doc.sentences for word in sent.words])
    words.append([word.text for sent in doc.sentences for word in sent.words])

1000
2000
3000
4000
5000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
65000


In [60]:
with open('pos_tags_dataset_sample.txt', 'w') as f:
    for line in pos_tags_dataset_sample:
        f.write(f"{str(line)}\n")

Writing the tags to a txt file (Simply)

Labelling the dataset

In [25]:
with open('/kaggle/input/pos-tags-txt/pos_tags_dataset_sample.txt','r') as f:
    first_3_lines = []
    for i in range(1):
        line = f.readline().strip()
        print(line)
        if line:
            first_3_lines.append(line)
res = re.findall(r'\[(.*?)\]', first_3_lines[0])


65694

In [35]:
res1 = res

In [36]:
for i in range(len(res)):
    substrings = res[i].split(", ")
    res[i] = [s.strip("'") for s in substrings]

In [47]:
len(dataset['comments'][0].split(' '))

14

In [40]:
len(dataset['comments'][1][:].split(' '))

28

In [44]:
len(pos_tags_dataset_sample)

1

In [19]:
len(dataset_sample['cleaned_comments'][2].split())

16

In [94]:
dataset_sample

,Unnamed: 0,comments,cleaned_comments,pos_tag,words
0,0,A lot of the clients I work with use SVB. This...,a lot of the clients i work with use svb this ...,"[DT, NN, IN, DT, NNS, PRP, VBP, IN, NN, NN, DT...","[a, lot, of, the, clients, i, work, with, use,..."
1,1,Remember when Jim Cramer a month ago said on h...,remember when jim cramer a month ago said on h...,"[VB, WRB, NNP, NNP, DT, NN, RB, VBD, IN, PRP$,...","[remember, when, jim, cramer, a, month, ago, s..."
2,2,"The FDIC only insures up to $250k, what happen...",the fdic only insures up to k what happens to ...,"[DT, NN, RB, VBZ, RP, TO, NN, WP, VBZ, IN, NNS...","[the, fdic, only, insures, up, to, k, what, ha..."
3,3,"This is a pretty big deal , they had a huge am...",this is a pretty big deal they had a huge amou...,"[DT, VBZ, DT, RB, JJ, NN, PRP, VBD, DT, JJ, NN...","[this, is, a, pretty, big, deal, they, had, a,..."
4,4,If anyone's wondering how a bank with 200+ bil...,if anyone s wondering how a bank with billion ...,"[IN, NN, VBZ, VBG, WRB, DT, NN, IN, CD, NNS, I...","[if, anyone, s, wondering, how, a, bank, with,..."
...,...,...,...,...,...
65689,65689,They already have enough local staff to fill t...,they already have enough local staff to fill t...,"[PRP, RB, VBP, JJ, JJ, NNS, TO, VB, DT, NNS, P...","[they, already, have, enough, local, staff, to..."
65690,65690,Indeed it is! It still is ridiculously below t...,indeed it is it still is ridiculously below th...,"[RB, PRP, VBZ, PRP, RB, VBZ, RB, IN, DT, VBN, ...","[indeed, it, is, it, still, is, ridiculously, ..."
65691,65691,Amazon did because it's their play for DoD con...,amazon did because it is their play for dod co...,"[NNP, VBD, IN, PRP, VBZ, PRP$, NN, IN, NN, NNS...","[amazon, did, because, it, is, their, play, fo..."
65692,65692,"Thanks, I didn't get past the paywall.",thanks i did not get past the paywall,"[NN, PRP, VBD, RB, VB, IN, DT, NN]","[thanks, i, did, not, get, past, the, paywall]"


In [20]:
dataset_sample['cleaned_comments'][0]

'a lot of the clients i work with use svb this should be interesting '

In [21]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [32]:
dataset_sample

,Unnamed: 0,comments,cleaned_comments
0,0,A lot of the clients I work with use SVB. This should be interesting…,a lot of the clients i work with use svb this should be interesting
1,1,Remember when Jim Cramer a month ago said on his show to buy it because it was cheap and had room to grow? Jimmy is such a meme.,remember when jim cramer a month ago said on his show to buy it because it was cheap and had room to grow jimmy is such a meme
2,2,"The FDIC only insures up to $250k, what happens to companies who have millions deposited there?",the fdic only insures up to k what happens to companies who have millions deposited there
3,3,"This is a pretty big deal , they had a huge amount of assets and deposits before the bad news started Perhaps that is why the government seized it before it can set off a domino effect across the banking sector",this is a pretty big deal they had a huge amount of assets and deposits before the bad news started perhaps that is why the government seized it before it can set off a domino effect across the banking sector
4,4,"If anyone's wondering how a bank with 200+ billion dollars in assets went belly up in less than two days, there were some [decent explanations in this OutOfTheLoop](https://www.reddit.com/r/OutOfTheLoop/comments/11n88pd/what_is_the_deal_with_silicon_valley_bank/) thread last night. One day you're doing ok, the next day the whole institution is just blown up. So weird how economics work...EDIT: mild correction on asset number",if anyone s wondering how a bank with billion dollars in assets went belly up in less than two days there were some [decent explanations in this outoftheloop] thread last night one day you are doing ok the next day the whole institution is just blown up so weird how economics work edit mild correction on asset number
...,...,...,...
65689,65689,They already have enough local staff to fill these buildings. They've been renting buildings/floors throughout the metro area to accommodate their growing staff since long before HQ2 became a thing. Now they'll just consolidate into Amazon-owned real estate.,they already have enough local staff to fill these buildings they have been renting buildings floors throughout the metro area to accommodate their growing staff since long before hq became a thing now they will just consolidate into amazon owned real estate
65690,65690,Indeed it is! It still is ridiculously below the promised value and hiring levels and Racine County is terrified Foxconn is just going to leave the moment they stop getting subsidies.,indeed it is it still is ridiculously below the promised value and hiring levels and racine county is terrified foxconn is just going to leave the moment they stop getting subsidies
65691,65691,"Amazon did because it's their play for DoD contracts. Remember this project started before the pandemic and the world is completely different now.As it relates to the tax credits, we'll have to see the fine print. Most of these deals have claw back provisions.",amazon did because it is their play for dod contracts remember this project started before the pandemic and the world is completely different now as it relates to the tax credits we will have to see the fine print most of these deals have claw back provisions
65692,65692,"Thanks, I didn't get past the paywall.",thanks i did not get past the paywall


In [84]:
dataset_sample.to_csv('pos_tags_dataset.csv', index=False)


In [35]:
len(pos_tags_dataset_sample[0])

2100506

In [82]:
dataset_sample['pos_tag'] = pos_tags_dataset

In [83]:
dataset_sample['words'] = words

In [27]:
dataset_sample.iloc[0,2:5]

cleaned_comments    a lot of the clients i work with use svb this ...
pos_tag             'DT', 'NN', 'IN', 'DT', 'NNS', 'PRP', 'VBP', '...
Name: 0, dtype: object

In [87]:
dataset_sample[['cleaned_comments', 'words']]

,cleaned_comments,words
0,a lot of the clients i work with use svb this ...,"[a, lot, of, the, clients, i, work, with, use,..."
1,remember when jim cramer a month ago said on h...,"[remember, when, jim, cramer, a, month, ago, s..."
2,the fdic only insures up to k what happens to ...,"[the, fdic, only, insures, up, to, k, what, ha..."
3,this is a pretty big deal they had a huge amou...,"[this, is, a, pretty, big, deal, they, had, a,..."
4,if anyone s wondering how a bank with billion ...,"[if, anyone, s, wondering, how, a, bank, with,..."
...,...,...
65689,they already have enough local staff to fill t...,"[they, already, have, enough, local, staff, to..."
65690,indeed it is it still is ridiculously below th...,"[indeed, it, is, it, still, is, ridiculously, ..."
65691,amazon did because it is their play for dod co...,"[amazon, did, because, it, is, their, play, fo..."
65692,thanks i did not get past the paywall,"[thanks, i, did, not, get, past, the, paywall]"


In [100]:
ls /kaggle/working/

__notebook_source__.ipynb  good_sentences.csv    pos_tags_dataset_sample.txt
bad_sentences.csv          pos_tags_dataset.csv


In [95]:
labels= []
word_indices=[]
for i,sentence in enumerate(dataset_sample['cleaned_comments']):
    flag = False
    for j,word in enumerate(dataset_sample['words'][i]):
        if word in bad_words_set:
            flag = True
            pos_tags_dataset[i][j] = 'BW'
    labels.append(1) if flag else labels.append(0)

In [56]:
len(res[1])

28

In [55]:
len(dataset_sample['cleaned_comments'][1].split())

28

Percentage of bad sentences

In [ ]:
np.count_nonzero(labels)

In [ ]:
np.count_nonzero(labels)/len(labels)

Separating good and bad sentences

In [96]:
dataset_sample['labels'] = labels

In [97]:
good_sentences = dataset_sample[dataset_sample['labels']==0]
bad_sentences = dataset_sample[dataset_sample['labels']==1]

In [101]:
good_sentences

,Unnamed: 0,comments,cleaned_comments,pos_tag,words,labels
0,0,A lot of the clients I work with use SVB. This...,a lot of the clients i work with use svb this ...,"[DT, NN, IN, DT, NNS, PRP, VBP, IN, NN, NN, DT...","[a, lot, of, the, clients, i, work, with, use,...",0
1,1,Remember when Jim Cramer a month ago said on h...,remember when jim cramer a month ago said on h...,"[VB, WRB, NNP, NNP, DT, NN, RB, VBD, IN, PRP$,...","[remember, when, jim, cramer, a, month, ago, s...",0
2,2,"The FDIC only insures up to $250k, what happen...",the fdic only insures up to k what happens to ...,"[DT, NN, RB, VBZ, RP, TO, NN, WP, VBZ, IN, NNS...","[the, fdic, only, insures, up, to, k, what, ha...",0
3,3,"This is a pretty big deal , they had a huge am...",this is a pretty big deal they had a huge amou...,"[DT, VBZ, DT, RB, JJ, NN, PRP, VBD, DT, JJ, NN...","[this, is, a, pretty, big, deal, they, had, a,...",0
4,4,If anyone's wondering how a bank with 200+ bil...,if anyone s wondering how a bank with billion ...,"[IN, NN, VBZ, VBG, WRB, DT, NN, IN, CD, NNS, I...","[if, anyone, s, wondering, how, a, bank, with,...",0
...,...,...,...,...,...,...
65689,65689,They already have enough local staff to fill t...,they already have enough local staff to fill t...,"[PRP, RB, VBP, JJ, JJ, NNS, TO, VB, DT, NNS, P...","[they, already, have, enough, local, staff, to...",0
65690,65690,Indeed it is! It still is ridiculously below t...,indeed it is it still is ridiculously below th...,"[RB, PRP, VBZ, PRP, RB, VBZ, RB, IN, DT, VBN, ...","[indeed, it, is, it, still, is, ridiculously, ...",0
65691,65691,Amazon did because it's their play for DoD con...,amazon did because it is their play for dod co...,"[NNP, VBD, IN, PRP, VBZ, PRP$, NN, IN, NN, NNS...","[amazon, did, because, it, is, their, play, fo...",0
65692,65692,"Thanks, I didn't get past the paywall.",thanks i did not get past the paywall,"[NN, PRP, VBD, RB, VB, IN, DT, NN]","[thanks, i, did, not, get, past, the, paywall]",0


In [98]:
len(good_sentences)
good_sentences.to_csv('good_sentences.csv')

In [104]:
bad_sentences['comments'].head()

7     dude. My employer uses SVB. We're dead in the ...
10    Yikes. All of our employees’ payroll direct de...
11    our company uses SVB. we have a pending wire o...
16    >Silicon Valley Bank, one of the tech sector’s...
28    If you or someone you know may be struggling w...
Name: comments, dtype: object

In [99]:
len(bad_sentences)
bad_sentences.to_csv('bad_sentences.csv')

In [ ]:
sample_bad = bad_sentences.sample(10)

In [ ]:
sample_bad

Lucene

In [ ]:
# import lucene
# from java.io import StringReader
# from org.apache.lucene.analysis.standard import StandardAnalyzer
# from org.apache.lucene.index import DirectoryReader
# from org.apache.lucene.search.similarities import ClassicSimilarity
# from org.apache.lucene.search import IndexSearcher
# from org.apache.lucene.queryparser.classic import QueryParser
# from org.apache.lucene.store import SimpleFSDirectory
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity

# # Initialize Lucene
# lucene.initVM()

# # Set up the analyzer and similarity algorithm
# analyzer = StandardAnalyzer()
# similarity = ClassicSimilarity()

# # Set up the index
# index_dir = SimpleFSDirectory(File("index"))
# searcher = IndexSearcher(DirectoryReader.open(index_dir))
# searcher.setSimilarity(similarity)

# # Define the query and tags
# query = "python programming"
# tags = ["python", "programming"]

# # Tokenize and vectorize the tags using TF-IDF
# tfidf = TfidfVectorizer(analyzer='word', stop_words='english')
# tfidf_matrix = tfidf.fit_transform(tags)

# # Search the index for similar sentences
# query_parser = QueryParser("content", analyzer)
# query = query_parser.parse(query)
# top_docs = searcher.search(query, 10)
# for score_doc in top_docs.scoreDocs:
#     doc = searcher.doc(score_doc.doc)
#     sentence = doc.get("content")
#     tfidf_score = cosine_similarity(tfidf_matrix, tfidf.transform([sentence]))[0][0]
#     if tfidf_score > 0.5:
#         print(f"Similar sentence found: {sentence}")

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Define the sentences
sentences = [
    # "The quick brown cat jumps over the lazy dog"
]

# Create a TfidfVectorizer object
vectorizer = TfidfVectorizer()

# Compute the tf-idf matrix
tfidf_matrix = vectorizer.fit_transform(sentences)

# Compute the cosine similarity matrix
cosine_sim_matrix = cosine_similarity(tfidf_matrix)

# Print the similarity matrix
print(cosine_sim_matrix)


[[1.         0.25861529 0.25861529]

 [0.25861529 1.         0.25861529]

 [0.25861529 0.25861529 1.        ]]


In [4]:
# with open('pos_tags_dataset_duplicate.txt','r') as f:
#     Lines = f.readlines()

#     # print(Lines[0])
#     lst = []
#     for i in range(len(Lines)):
#         lst.append(list(Lines[i]))

'NN', 'DT', 'JJ', 'NN', 'IN', 'NNP', 'CC', 'NNP', 'VBZ', 'VBG', 'IN', 'IN', 'NN', '.'


